# Customer Segmentation
Customer segmentation is a method to segment customers in order to customize business strategy to different segment of customer. The idea is appear due to each customer has to be treated in different way to another. This also useful to understand what kind of customer do we need to prioritize and maintain.

#### 1. Importing Libraries and BigQuery Setup

In [7]:
import numpy as np
import pandas as pd
import pandas_gbq as pbq
from google.oauth2 import service_account
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans

# tracking BigQuery usage during execution
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

# setup credetials (not available in GitHub as it is "credentials")
credentials = service_account.Credentials.from_service_account_file('../../main_v2_key.json')

# the id of GCP project
project_id = 'dw-production'

#### 2. Pareto Principle
Other factor that also related to this customer segmentation is Pareto Priciple or 80/20 rule. Pareto Principle is basicly working on a lot of things and one of them is business customers. Using Pareto Principle we can say that 20% of customers cause 80% of revenue/sales/deliveries and so on (what ever our metrics is). This Pareto Principle also works on our dataset.

##### a. Load Dataset

In [4]:
pareto_query = """
SELECT user_id,
       SUM(sold) AS sold,
       MAX(total) AS total
FROM
(SELECT CASE 
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 5 THEN 5
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 10 THEN 10
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 15 THEN 15
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 20 THEN 20
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 25 THEN 25
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 30 THEN 30
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 35 THEN 35
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 40 THEN 40
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 45 THEN 45
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 50 THEN 50
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 55 THEN 55
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 60 THEN 60
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 65 THEN 65
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 70 THEN 70
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 75 THEN 75
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 80 THEN 80
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 85 THEN 85
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 90 THEN 90
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 95 THEN 95
         WHEN 100*(ROW_NUMBER() OVER(ORDER BY sold DESC))/(COUNT(DISTINCT consumer_id) OVER()) <= 100 THEN 100
       END AS user_id,
        sold,
       100*(SUM(sold) OVER(ORDER BY sold DESC))/(SUM(sold) OVER()) AS total
FROM
  (SELECT o.consumer_id AS consumer_id,
          SUM(CAST(o.sd_qty AS NUMERIC)) AS sold
   FROM main_v2.dim_orders_2019 AS o
   WHERE o.status = 1
     AND DATE(o.order_date) BETWEEN DATE(EXTRACT(YEAR FROM CURRENT_DATE()),1,1) AND CURRENT_DATE()
   GROUP BY o.consumer_id)
ORDER BY sold DESC)
GROUP BY 1
"""

df_pareto = pbq.read_gbq(pareto_query, project_id, credentials=credentials)

Requesting query... 
Query running...
Job ID: 44ab15cb-23aa-45dc-97c0-ae3657e1ad18
Query done.
Processed: 9.0 MB Billed: 10.0 MB
Standard price: $0.00 USD

Got 20 rows.



In [5]:
df_pareto.head()

,user_id,sold,total
0,5,128400,35.581290048
1,10,51354,49.525665125
2,15,36620,59.863064448
3,20,28036,67.683209616
4,25,22011,74.27697205


##### b. Visualize using Combo Chart

In [10]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_pareto.user_id,y=df_pareto.sold,name="Deliveries"), secondary_y=False)
fig.add_trace(go.Scatter(x=df_pareto.user_id,y=df_pareto.total,name="Total(%)"), secondary_y=True)
fig.update_layout(title_text="Customer Deliveries Combo Chart")
fig.update_xaxes(title_text="Customer Ranking (%)")
fig.update_yaxes(title_text="Deliveries", secondary_y=False)
fig.update_yaxes(title_text="Total Deliveries (%)", secondary_y=True)
fig.show()

The graph shows that top 25% of customers affect almost 75% of deliveries, it means by focusing our business to this 25% of our customer segment we can at least maintain and retain 75% of our deliveries and if some new customers acquired with the same profile as this segment, the deliveries can be increased dramatically while the current retention of this segment also maintained.  

#### 3. RFM Clustering
The customer segmentation method vary on different metric we are focused on, but there is a customer segmentation method that can be used in common, it is called RFM (Recency, Frequency and Monetery). This method has developed and derived into different method, the latest developement of this method is called RFMTC (Recency, Frequency, Monetery, Time and Churn rate) but this notebook will cover the original RFM using k-means clustering analysis. k-means cluster later can be used as a score. So, we have to make sure that cluster 0 is the worst customer segment and cluster n-1 is the best customer.

##### a. Recency
Recency is technically how recent the customer active in using our services. Intuitively, the best recency is the lowest day difference compared to current date, so we need to find the customer segment who has lowest recency. to find recency the formula is quite obvious date_diff(current_date(), last_transaction_date).
##### b. Frequency
Frequency also obvious, it's about how much the customer use our service. In the available dataset, the usage of our service is calculated by the number of order was made by the customer, so it is clear that is highest cluster should be a place for the highest frequency
##### c. Monetery (Total Spend)
In customer point of view, the monetery in here is the total amount of money that they have spent to our services. and it also obvious that the highest spending will be placed in the highest cluster.
####  d. Overall
The overall score of the customer will define in which segment does the customer placed. overall score simply the sum of all three score above. So, it shouldn't be confusing.

#### 4. Implementation
The implementation is quite straight forward, so let's jump into it.

##### a. Load Dataset

In [16]:
query = """
SELECT o.consumer_id,
       DATE_DIFF(CURRENT_DATE(), DATE(MAX(o.order_date)), DAY) AS recency,
       COUNT(DISTINCT o.id) AS frequency,
       SUM(tp.amount) AS monetery
FROM main_v2.dim_orders_2019 AS o
JOIN main_v2.dim_topup_payments AS tp ON tp.subscription_id = o.subscription_id
WHERE o.status = 1
  AND tp.status = "PAID"
  AND tp.amount > 0
GROUP BY o.consumer_id
"""

df = pbq.read_gbq(query, project_id, credentials=credentials)

Requesting query... 
Requesting query... 
Query running...
Query running...
Job ID: d94b7b6d-9f1f-4ccb-a54a-98aec4dff5b3
Job ID: d94b7b6d-9f1f-4ccb-a54a-98aec4dff5b3
Query done.
Processed: 17.0 MB Billed: 20.0 MB
Query done.
Processed: 17.0 MB Billed: 20.0 MB
Standard price: $0.00 USD

Standard price: $0.00 USD

Got 19454 rows.

Got 19454 rows.



In [17]:
df.head()

,consumer_id,recency,frequency,monetery
0,77232,31,61,13358900
1,86651,76,27,25475900
2,84302,60,19,17147500
3,61403,19,27,8975500
4,91575,10,20,5726400


In [18]:
df.recency.describe()

count    19454.000000
mean       117.434564
std         78.128579
min          6.000000
25%         51.000000
50%        107.000000
75%        175.000000
max        291.000000
Name: recency, dtype: float64

In [19]:
df.frequency.describe()

count    19454.000000
mean         4.043127
std          6.014773
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max        132.000000
Name: frequency, dtype: float64

In [20]:
df.monetery.describe()

count    1.945400e+04
mean     2.755025e+06
std      1.148908e+07
min      2.000000e+02
25%      1.863125e+05
50%      7.150000e+05
75%      2.000000e+06
max      8.716680e+08
Name: monetery, dtype: float64

##### b. Pick the best number of clusters

In [34]:
sse = dict()
df_factors = df[['recency', 'frequency', 'monetery']]
for k in range(1, 30):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(df_factors)
    df_factors["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(sse.keys()),y=list(sse.values())))
fig.show()

Looks like 10 clusters or above already stable, but to simplify all the things let start with 10 clusters 